# Solucion Guía 3

In [1]:
%pip install --quiet openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


In [3]:
# 5) Primera consulta: respuesta libre
prompt = "Explica en dos frases qué es la inteligencia artificial en la educación."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7  # más creativo que 0.2, menos que 1.0
)
print(response.choices[0].message.content)

La inteligencia artificial en la educación se refiere al uso de tecnologías avanzadas para personalizar el aprendizaje, analizar el rendimiento de los estudiantes y mejorar la enseñanza a través de herramientas como tutores virtuales y plataformas adaptativas. Estas soluciones pueden facilitar una experiencia educativa más efectiva y accesible, adaptándose a las necesidades individuales de cada alumno.


In [4]:
# 6) Respuesta estructurada en JSON para automatización
import json

query = "¿Qué es aprendizaje supervisado?"
schema_instruction = (
    "Responde en formato JSON con las claves: operation, input, output. "
    "operation debe ser 'explanation'; input debe repetir la pregunta; output la explicación clara y breve."
)
response_json = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": query}
    ],
    temperature=0.3,       # más determinista para formatos estructurados
    max_tokens=300         # suficiente para una explicación breve
)
text = response_json.choices[0].message.content
print(text)

# (Opcional) intentar cargar como JSON si el modelo devolvió un objeto válido
try:
    data = json.loads(text)
    print("\nValid JSON →", data)
except json.JSONDecodeError:
    print("\nLa salida no es JSON válido literal. Puedes parsearla manualmente o usar validadores/funciones JSON del proveedor.")

{
  "operation": "explanation",
  "input": "¿Qué es aprendizaje supervisado?",
  "output": "El aprendizaje supervisado es un tipo de técnica de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada en el conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida para nuevas entradas basándose en patrones encontrados en los datos de entrenamiento."
}

Valid JSON → {'operation': 'explanation', 'input': '¿Qué es aprendizaje supervisado?', 'output': 'El aprendizaje supervisado es un tipo de técnica de aprendizaje automático donde un modelo es entrenado utilizando un conjunto de datos etiquetados. Esto significa que cada entrada en el conjunto de datos tiene una salida correspondiente conocida, lo que permite al modelo aprender a predecir la salida para nuevas entradas basándose en patrones encontrados en los datos de entrenamiento.'}


## 7) Ejercicios propuestos
1. Cambia `temperature` a 0.1, 0.5 y 0.9 y compara el estilo de las respuestas.
2. Pide que el modelo responda **siempre en JSON** usando un `system` que lo exija. Verifica si cumple.
3. Crea un prompt de tu área (p.ej., *programación*, *arquitectura*, *matemáticas*) que devuelva un JSON con `operation`, `input`, `steps` (lista) y `output`.
4. Limita la longitud con `max_tokens` y observa si corta.

In [5]:
# Temperature = 0.1
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.1,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "promt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de predicción que utiliza una estructura jerárquica de nodos para dividir un conjunto de datos en subgrupos basados en características específicas. Cada nodo representa una pregunta sobre una característica, y las ramas indican las posibles respuestas, llevando a hojas que representan la decisión final o la predicción."
}


In [6]:
# Temperature = 0.5, tokens = 200
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.5,
              max_tokens: int = 200,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

# Ejemplo de uso
print(ask_model(
    "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
    temperature=0.4,
    system="Responde en dos oraciones, tono docente y preciso. Siempre responde JSON con llaves promt y respuesta"
))

{
  "prompt": "Explica brevemente el principio de funcionamiento de un árbol de decisión.",
  "respuesta": "Un árbol de decisión es un modelo de aprendizaje automático que utiliza una estructura jerárquica para tomar decisiones basadas en características de los datos. Cada nodo interno representa una prueba sobre un atributo, cada rama representa el resultado de esa prueba, y cada hoja representa una clase o resultado final, permitiendo clasificar o predecir datos de manera intuitiva."
}


In [10]:
def ask_model(prompt: str,
              model: str = "gpt-4o-mini",
              temperature: float = 0.9,
              max_tokens: int = 400,
              system: str | None = None):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content

print(ask_model(
    "Crea un script basico de python que ordene una lista de números",
    temperature=0.3,
    system="Responde siempre en JSON con las claves: operation, input, steps, output."
))


```json
{
  "operation": "create_script",
  "input": {
    "language": "python",
    "task": "sort a list of numbers"
  },
  "steps": [
    "Define a list of numbers.",
    "Use the built-in sorted() function to sort the list.",
    "Print the sorted list."
  ],
  "output": "script.py",
  "script": "numbers = [5, 2, 9, 1, 5, 6]\nsorted_numbers = sorted(numbers)\nprint(sorted_numbers)"
}
```
